In [139]:
import trimesh
import mesh_raycast
# 读取OBJ文件
obj_file_path = 'E:\\code\\ObjectLocation\\data\\Desert.obj'
mesh = trimesh.load_mesh(obj_file_path)
# real_point = np.array([-6.366050, -30.999302 ,-0.982937 ]).reshape(3,1)
# rotation_matrix = R3
# rotation_matrix_inv = np.linalg.inv(rotation_matrix)
# translation_vector= np.array(t3).reshape(3,1)
# K_inv = np.linalg.inv(K)
# p_cam = rotation_matrix_inv @ (real_point-translation_vector)
# print("p_cam" ,p_cam)
# p_cam = p_cam / p_cam[2]
# pixel_point = K @ p_cam
# print("p_cam" ,p_cam)
# print("pixel" ,pixel_point)

# p_c1 = np.dot(K_inv, pixel_point)
# print('p_c1',rotation_matrix @ p_c1*9.04705184 + translation_vector)
# # 归一化  o+t1d, 转到世界坐标系R(O+t1d)+t = t1*Rd + t
# pc_d = p_c1 / np.linalg.norm(p_c1)
# pw_d = np.dot(rotation_matrix, pc_d)
# ray_origins = translation_vector
# ray_end =  ray_origins + pw_d*20 # 3*1
# ray_directions = pw_d.flatten()


# # raycast方法二：
# triangles = list(mesh.geometry.values())[0].vertices[list(mesh.geometry.values())[0].faces]
# triangles = np.array(triangles, dtype='f4')  # 一定要有这一行，不然会有错。
# result = mesh_raycast.raycast(ray_origins, ray_directions, mesh=triangles)
# print(result)
# first_result = min(result, key=lambda x: x['distance'])
# print("result", first_result['point'])
# print("real_point",real_point.flatten())



# 怎么得到mesh上的z坐标
ray_origins = [-6.366050, -30.999302 ,-20 ]
ray_directions = [0.,0,1]
triangles = list(mesh.geometry.values())[0].vertices[list(mesh.geometry.values())[0].faces]
triangles = np.array(triangles, dtype='f4')  # 一定要有这一行，不然会有错。
result = mesh_raycast.raycast(ray_origins, ray_directions, mesh=triangles)
first_result = min(result, key=lambda x: x['distance'])
print("result", first_result['point'])


ValueError: min() iterable argument is empty

In [143]:
import json
# 单相机单时刻2跟踪

def create_json_file(timestamp, uav_id, camera_id, pose, K, distortion, tracked_data):
    data = {
        "timestamp": timestamp,
        "uav_id": uav_id,
        "camera_id": camera_id,
        "camera_params": {
            "pose": pose,  # [yaw,pitch,roll,x,y,z]
            "K": K,        # [fx,fy,cx,cy]
            "distortion": distortion  # [k1,k2,p1,p2]
        },
        "obj_cnt": len(tracked_data),
        "objs": [],
        "obj_img": None
    }

    for tracked_item in tracked_data:
        obj = {
            "tracker_id": tracked_item[0],
            "cls_id": tracked_item[1],  # 0车1人
            "bbox": tracked_item[2],    # [x,y,w,h]
            "loc": tracked_item[3]      # (WGS84）
        }
        data["objs"].append(obj)

    # Writing data to JSON file
    with open('jsons/time1.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)

timestamp = 124567824564654
uav_id = 1
camera_id = 1
# [yaw, pitch, roll, x, y ,z]
pose = [0., 0., 150., 0., -25., 6.]   
# [fx, fy, cx, cy]
K = [355.72670241928597, 357.6787245904993, 311.9712774887887, 253.00946170247045] 
# [k1,k2,p1,p2]
distortion = [-0.16511311963465314, 0.06499598603806166, 0.0033307309789138034, 0.007443839439139966]
# [tracker_id, cls_id, bbox, loc]
traked_num = 2
tracked_data = [[1,0,[30,30,3,4],[]],[2,1,[20,30,1,2],[]]]

create_json_file(timestamp, uav_id, camera_id, pose, K, distortion, tracked_data)

In [149]:
import json
# 单相机多时刻2目标跟踪

def create_json_file(timestamp, uav_id, camera_id, pose, K, distortion, tracked_data, i):
    data = {
        "timestamp": timestamp,
        "uav_id": uav_id,
        "camera_id": camera_id,
        "camera_params": {
            "pose": pose,  # [yaw,pitch,roll,x,y,z]
            "K": K,        # [fx,fy,cx,cy]
            "distortion": distortion  # [k1,k2,p1,p2]
        },
        "obj_cnt": len(tracked_data),
        "objs": [],
        "obj_img": None
    }

    for tracked_item in tracked_data:
        obj = {
            "tracker_id": tracked_item[0],
            "cls_id": tracked_item[1],  # 0车1人
            "bbox": tracked_item[2],    # [x,y,w,h]
            "loc": tracked_item[3]      # (WGS84）
        }
        data["objs"].append(obj)

    # Writing data to JSON file
    with open('jsons/time'+str(i)+'.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)

timestamp = 124567824564654
uav_id = 1
camera_id = 1
# [yaw, pitch, roll, x, y ,z]
pose = [0., 0., 150., 0., -25., 6.]   
# [fx, fy, cx, cy]
K = [355.72670241928597, 357.6787245904993, 311.9712774887887, 253.00946170247045] 
# [k1,k2,p1,p2]
distortion = [-0.16511311963465314, 0.06499598603806166, 0.0033307309789138034, 0.007443839439139966]
# [tracker_id, cls_id, bbox, loc]
traked_num = 2
tracked_data = [[1,0,[30,30,3,4],[]],[2,1,[20,30,1,2],[]]]
time = 10
for i in range(time):
    timestamp = timestamp + 1
    for j in range(len(tracked_data)):
        # 像素位置更新
        if tracked_data[j][1] == 0: # 车，每次移动4像素
            tracked_data[j][2][0] =  tracked_data[j][2][0] + 4
            tracked_data[j][2][1] =  tracked_data[j][2][1] + 4
        elif tracked_data[j][1] == 1: # 人，每次移动1像素
            tracked_data[j][2][0] =  tracked_data[j][2][0] + 2
            tracked_data[j][2][1] =  tracked_data[j][2][1] + 2     
    create_json_file(timestamp, uav_id, camera_id, pose, K, distortion, tracked_data, i)

In [152]:
# json转为class package
import json

class Package:
    def __init__(self, time):

        # Public variables
        # read only
        self.time = time
        self.uav_id: int = None
        self.camera_pose: list[float] = [] # [yaw,pitch,roll,x,y,z]
        self.camera_K: list[float] = [] # [fx,fy,cx,cy]
        self.camera_distortion: list[float] = [] #[k1,k2,p1,p2]
        self.Bbox: list[int] = []
        self.class_id: int = None
        self.class_name: str = None
        self.tracker_id: int = None
        self.uav_pos: list[float] = []
        self.obj_img: str = None
        # read & write
        self.global_id: int = None
        self.local_id: int = None
        self.location: list[float] = []

    def get_center_point(self) -> list[float]:
        return [(self.Bbox[0]+self.Bbox[2])/2, (self.Bbox[1]+self.Bbox[3])/2] # TODO 有错误

    def __str__(self):
        return f"time:{self.time}"
    
    # def display_info(self):
    #     print(f"Time: {self.time}")
    #     print(f"UAV ID: {self.uav_id}")
    #     print(f"Camera Pose: {self.camera_pose}")
    #     print(f"Camera K: {self.camera_K}")
    #     print(f"Camera Distortion: {self.camera_distortion}")
    #     print(f"Bbox: {self.Bbox}")
    #     print(f"Class ID: {self.class_id}")
    #     print(f"Class Name: {self.class_name}")
    #     print(f"Tracker ID: {self.tracker_id}")
    #     print(f"UAV Position: {self.uav_pos}")
    #     print(f"Object Image: {self.obj_img}")
    #     print(f"Global ID: {self.global_id}")
    #     print(f"Local ID: {self.local_id}")
    #     print(f"Location: {self.location}")

def parse_json_to_packages(file_path):
    packages = []

    with open(file_path, 'r') as file:
        data = json.load(file)

        # 解析JSON数据并创建Package对象
        for obj in data['objs']:
            package = Package(data['timestamp'])

            package.uav_id = data['uav_id']
            package.camera_pose = data['camera_params']['pose']
            package.camera_K = data['camera_params']['K']
            package.camera_distortion = data['camera_params']['distortion']
            package.Bbox = obj['bbox']
            package.class_id = obj['cls_id']
            package.tracker_id = obj['tracker_id']
            package.obj_img = data['obj_img']

            # 可以根据需要设置其他属性...

            packages.append(package)

    return packages

# 示例路径
file_path = 'jsons/time1.json'

# 解析JSON文件并创建Package对象列表
packages = parse_json_to_packages(file_path)

# 打印每个Package对象的信息
# for package in packages:
#     package.display_info()


Time: 124567824564656
UAV ID: 1
Camera Pose: [0.0, 0.0, 150.0, 0.0, -25.0, 6.0]
Camera K: [355.72670241928597, 357.6787245904993, 311.9712774887887, 253.00946170247045]
Camera Distortion: [-0.16511311963465314, 0.06499598603806166, 0.0033307309789138034, 0.007443839439139966]
Bbox: [38, 38, 3, 4]
Class ID: 0
Class Name: None
Tracker ID: 1
UAV Position: []
Object Image: None
Global ID: None
Local ID: None
Location: []
Time: 124567824564656
UAV ID: 1
Camera Pose: [0.0, 0.0, 150.0, 0.0, -25.0, 6.0]
Camera K: [355.72670241928597, 357.6787245904993, 311.9712774887887, 253.00946170247045]
Camera Distortion: [-0.16511311963465314, 0.06499598603806166, 0.0033307309789138034, 0.007443839439139966]
Bbox: [24, 34, 1, 2]
Class ID: 1
Class Name: None
Tracker ID: 2
UAV Position: []
Object Image: None
Global ID: None
Local ID: None
Location: []
